In [21]:
import pandas as pd

# --- load ---
Bteselem_0011 = pd.read_csv("2000-2011_bteselem.csv")
Bteselem_1219 = pd.read_csv("2011-2020_bteselem.csv")

# --- rename date column to a common name ---
Bteselem_0011 = Bteselem_0011.rename(columns={"Date of event": "Date"})
Bteselem_1219 = Bteselem_1219.rename(columns={"Date of event_R": "Date"})

# --- parse Date ---
Bteselem_0011["Date"] = pd.to_datetime(Bteselem_0011["Date"])
Bteselem_1219["Date"] = pd.to_datetime(Bteselem_1219["Date"])

# --- unify names across the two files BEFORE concat ---
Bteselem_0011 = Bteselem_0011.rename(columns={
    "occurence_R": "occurrence_R",
})

Bteselem_1219 = Bteselem_1219.rename(columns={
    "occurence_R": "occurrence_R",
    "occurrence_pal_t_y": "occurrence_pal_t_y_R",
})

# Drop time cols from 1219 (we rebuild later at master merge)
drop_time_cols = ["month_R", "year_R", "dow_R", "monthyear_R"]
Bteselem_1219 = Bteselem_1219.drop(columns=[c for c in drop_time_cols if c in Bteselem_1219.columns])

# IMPORTANT: ensure high_intensity column exists in BOTH before concat
# (so 0011 won't become NaN)
if "gaza_war_R" not in Bteselem_0011.columns:
    Bteselem_0011["gaza_war_R"] = 0

# --- concat ---
Bteselem_0019 = pd.concat([Bteselem_0011, Bteselem_1219], ignore_index=True)

# --- rename variables according to the paper ---
Bteselem_0019 = Bteselem_0019.rename(columns={
    # occurrence & victims
    "occurrence_R": "occurrence",
    "occurrence_pal_R": "occurrence_pal",
    "victims_isr_R": "victims_isr",
    "victims_pal_R": "victims_pal",
    "lnvic_R": "lnvic",
    "lnvic_pal_R": "lnvic_pal",

    # conflict intensity
    "gaza_war_R": "high_intensity",

    # t/y style variables
    "lnvic_t_y_R": "lnvic_t_y",
    "lnvic_pal_y_R": "lnvic_pal_y",
    "occurrence_t_y_R": "occurrence_t_y",
    "occurrence_pal_t_y_R": "occurrence_pal_t_y",

    # lags/windows
    "occurrence_1_R": "occurrence_1",
    "occurrence_2_7_R": "occurrence_2_7",
    "occurrence_8_14_R": "occurrence_8_14",
    "occurrence_pal_1_R": "occurrence_pal_1",
    "occurrence_pal_2_7_R": "occurrence_pal_2_7",
    "occurrence_pal_8_14_R": "occurrence_pal_8_14",
})

# --- your request: when not high intensity, use 0 not NaN ---
Bteselem_0019["high_intensity"] = (
    Bteselem_0019["high_intensity"]
    .fillna(0)
    .astype(float)
)

# Force strictly 0/1 (in case there are weird values)
Bteselem_0019["high_intensity"] = (Bteselem_0019["high_intensity"] > 0).astype(int)

# quick check
print("high_intensity counts:")
print(Bteselem_0019["high_intensity"].value_counts(dropna=False))

print("Date range:", Bteselem_0019["Date"].min(), "to", Bteselem_0019["Date"].max())
print("Shape:", Bteselem_0019.shape)
Bteselem_0019["high_intensity"].value_counts()

# make sure Date is datetime
Bteselem_0019["Date"] = pd.to_datetime(Bteselem_0019["Date"])

# start with all zeros
Bteselem_0019["high_intensity"] = 0

# High-intensity episodes (inclusive start/end)
episodes = [
    # Cast Lead
    ("2008-12-27", "2009-01-18"),
    # Pillar of Defense
    ("2012-11-14", "2012-11-21"),
    # Protective Edge
    ("2014-07-08", "2014-08-26"),
]

for start, end in episodes:
    mask = (Bteselem_0019["Date"] >= start) & (Bteselem_0019["Date"] <= end)
    Bteselem_0019.loc[mask, "high_intensity"] = 1

# (optional) quick sanity checks
print(Bteselem_0019["high_intensity"].value_counts(dropna=False))
print(Bteselem_0019.loc[Bteselem_0019["high_intensity"] == 1, "Date"].agg(["min", "max", "count"]))
Bteselem_0019.groupby(Bteselem_0019["Date"].dt.year)["high_intensity"].sum()


high_intensity counts:
high_intensity
0    7341
1      58
Name: count, dtype: int64
Date range: 2000-09-29 00:00:00 to 2020-12-31 00:00:00
Shape: (7399, 18)
high_intensity
0    7318
1      81
Name: count, dtype: int64
min      2008-12-27 00:00:00
max      2014-08-26 00:00:00
count                     81
Name: Date, dtype: object


Date
2000     0
2001     0
2002     0
2003     0
2004     0
2005     0
2006     0
2007     0
2008     5
2009    18
2010     0
2011     0
2012     8
2013     0
2014    50
2015     0
2016     0
2017     0
2018     0
2019     0
2020     0
Name: high_intensity, dtype: int64

In [22]:
#importation of US TV News data
TV_news = pd.read_csv("vandy_daily_pressure_2000_2025_colonnes_clean.csv")
TV_news.head()


,Date,program_title,station,daily_woi,all_nonconf_min,daily_woi_nc,daily_woi_c,any_conflict_news,length_conflict_news,num_conflict_news
0,2000-01-03,ABC Evening News,ABC,5.166667,25.833333,5.166667,6.000000,1.0,1.833333,18.0
1,2000-01-03,CBS Evening News,CBS,5.500000,26.666667,5.500000,6.187500,1.0,1.833333,23.0
2,2000-01-04,ABC Evening News,ABC,5.833333,27.500000,5.833333,6.363636,1.0,0.333333,21.0
3,2000-01-04,CBS Evening News,CBS,6.333333,27.500000,6.333333,6.909091,1.0,0.333333,22.0
4,2000-01-05,ABC Evening News,ABC,7.333333,27.833333,7.333333,7.904192,0.0,0.000000,14.0


In [23]:
import pandas as pd
import numpy as np

TV_news = pd.read_csv("vandy_daily_pressure_2000_2025_colonnes_clean.csv")

# clean column names + parse date
TV_news.columns = TV_news.columns.str.strip()
TV_news["Date"] = pd.to_datetime(TV_news["Date"])

# agrégation à 1 ligne / date
TV_day = (
    TV_news
    .groupby("Date", as_index=False)
    .agg(
        daily_woi=("daily_woi", "median"),
        daily_woi_nc=("daily_woi_nc", "median"),
        daily_woi_c=("daily_woi_c", "median"),           # si tu l'utilises
        any_conflict_news=("any_conflict_news", "max"),
        length_conflict_news=("length_conflict_news", "sum"),
        num_conflict_news=("num_conflict_news", "sum"),
        all_nonconf_min=("all_nonconf_min", "median"),  # optionnel
    )
)

TV_day.head()

TV_day.shape
#TV_day["Date"].min(), TV_day["Date"].max()
full_dates = pd.date_range(
    start=TV_day["Date"].min(),
    end=TV_day["Date"].max(),
    freq="D"
)
missing_dates = full_dates.difference(TV_day["Date"])
len(missing_dates)
pd.DataFrame({"Date": missing_dates}).head(20)




,Date
0,2000-01-08
1,2000-01-29
2,2000-02-01
3,2000-02-02
4,2000-02-03
5,2000-02-04
6,2000-02-05
7,2000-02-06
8,2000-02-07
9,2000-02-08


In [24]:
#import data on major events
lead_maj_events = pd.read_csv("data_clean/major_events_dummy_2000_2019.csv")
lead_maj_events.head()

,Date,major_political_event,fifa_event,lead_maj_events
0,2000-01-01,0,0,0
1,2000-01-02,0,0,0
2,2000-01-03,0,0,0
3,2000-01-04,0,0,0
4,2000-01-05,0,0,0


In [25]:
#import data on disaster event
lead_disasters = pd.read_csv("data_clean/disaster_daily_2000_2019.csv")
lead_disasters.head()

,Date,lead_disaster
0,2000-01-01,0
1,2000-01-02,0
2,2000-01-03,0
3,2000-01-04,0
4,2000-01-05,0


In [26]:
#import data on google searches 
conflict_searches = pd.read_csv("google_searches_2000_2020.csv")
#note: ce dataset comment en 2004
conflict_searches = conflict_searches.rename(columns={"date_R": "Date"})
conflict_searches.head()

,Date,conflict_searches_R
0,2000-09-29,NaN
1,2000-09-30,NaN
2,2000-10-01,NaN
3,2000-10-02,NaN
4,2000-10-03,NaN


In [27]:
#merge tout ensemble
datasets = [
    conflict_searches,
    lead_disasters,
    lead_maj_events,
    TV_day,
    Bteselem_0019
]

for df in datasets:
    df.columns = df.columns.str.strip()
    df["Date"] = pd.to_datetime(df["Date"])

df.head()
print(df.columns)


Index(['Date', 'victims_isr', 'occurrence', 'victims_pal', 'occurrence_pal',
       'lnvic', 'lnvic_pal', 'lnvic_t_y', 'lnvic_pal_y', 'occurrence_t_y',
       'occurrence_pal_t_y', 'occurrence_1', 'occurrence_2_7',
       'occurrence_8_14', 'occurrence_pal_1', 'occurrence_pal_2_7',
       'occurrence_pal_8_14', 'high_intensity'],
      dtype='object')


In [28]:
#include the month, year, dow, monthyear variables 

df["month"] = df["Date"].dt.month.astype("float32")
df["year"] = df["Date"].dt.year.astype("float32")
df["dow"] = (
    df["Date"]
    .dt.day_name()
    .astype("category")
)
dow_order = [
    "Sunday", "Monday", "Tuesday", "Wednesday",
    "Thursday", "Friday", "Saturday"
]

df["dow"] = pd.Categorical(
    df["dow"],
    categories=dow_order,
    ordered=True
)
df["monthyear"] = (
    df["Date"]
    .dt.to_period("M")
    .astype(str)
)

df["month"].head(15)
df["year"].head(15)
df["dow"].head(15)

df.columns

Index(['Date', 'victims_isr', 'occurrence', 'victims_pal', 'occurrence_pal',
       'lnvic', 'lnvic_pal', 'lnvic_t_y', 'lnvic_pal_y', 'occurrence_t_y',
       'occurrence_pal_t_y', 'occurrence_1', 'occurrence_2_7',
       'occurrence_8_14', 'occurrence_pal_1', 'occurrence_pal_2_7',
       'occurrence_pal_8_14', 'high_intensity', 'month', 'year', 'dow',
       'monthyear'],
      dtype='object')

In [30]:
import pandas as pd

# 1) Standardize column names + date dtype
def prep(df):
    df = df.copy()
    df.columns = df.columns.str.strip()

    # accept both "Date" or "date", standardize to "date"
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
    elif "Date" in df.columns:
        df = df.rename(columns={"Date": "date"})
        df["date"] = pd.to_datetime(df["date"])
    else:
        raise KeyError("No date column found (expected 'Date' or 'date').")

    return df

conflict_searches = prep(conflict_searches).rename(columns={"conflict_searches_R": "conflict_searches"})
lead_disasters    = prep(lead_disasters).rename(columns={"lead_disaster_R": "lead_disaster"})
lead_maj_events   = prep(lead_maj_events).rename(columns={"lead_maj_events_R": "lead_maj_events"})
TV_day            = prep(TV_day)
Bteselem_0019     = prep(Bteselem_0019)

# 2) Ensure each is 1 row per date
def assert_unique_date(df, name):
    if not df["date"].is_unique:
        dup = df[df["date"].duplicated(keep=False)].sort_values("date")
        raise ValueError(f"{name} has duplicate dates. Example:\n{dup.head(10)}")

assert_unique_date(conflict_searches, "conflict_searches")
assert_unique_date(lead_disasters, "lead_disasters")
assert_unique_date(lead_maj_events, "lead_maj_events")
assert_unique_date(TV_day, "TV_day")
assert_unique_date(Bteselem_0019, "Bteselem_0019")

# 3) Merge (base = Bteselem)
df_master = (
    Bteselem_0019
    .merge(TV_day, on="date", how="left")
    .merge(conflict_searches, on="date", how="left")
    .merge(lead_maj_events, on="date", how="left")
    .merge(lead_disasters, on="date", how="left")
)

# 4) Add time vars
df_master["month"] = df_master["date"].dt.month.astype("float32")
df_master["year"]  = df_master["date"].dt.year.astype("float32")

df_master["dow"] = df_master["date"].dt.day_name()
dow_order = ["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
df_master["dow"] = pd.Categorical(df_master["dow"], categories=dow_order, ordered=True)

df_master["monthyear"] = df_master["date"].dt.to_period("M").astype(str)

print(df_master.shape)
print(df_master.columns)

# 5) Save
out_path = "data_clean/full_dataset_cleaned.csv"
df_master.to_csv(out_path, index=False)

# quick read check
print(pd.read_csv(out_path).head())

# sanity checks
print("date range:", df_master["date"].min(), "to", df_master["date"].max())
print("unique dates:", df_master["date"].is_unique)


(7399, 34)
Index(['date', 'victims_isr', 'occurrence', 'victims_pal', 'occurrence_pal',
       'lnvic', 'lnvic_pal', 'lnvic_t_y', 'lnvic_pal_y', 'occurrence_t_y',
       'occurrence_pal_t_y', 'occurrence_1', 'occurrence_2_7',
       'occurrence_8_14', 'occurrence_pal_1', 'occurrence_pal_2_7',
       'occurrence_pal_8_14', 'high_intensity', 'month', 'year', 'dow',
       'monthyear', 'daily_woi', 'daily_woi_nc', 'daily_woi_c',
       'any_conflict_news', 'length_conflict_news', 'num_conflict_news',
       'all_nonconf_min', 'conflict_searches', 'major_political_event',
       'fifa_event', 'lead_maj_events', 'lead_disaster'],
      dtype='object')
         date  victims_isr  occurrence  victims_pal  occurrence_pal     lnvic  \
0  2000-09-29          5.0         1.0          1.0             1.0  1.791759   
1  2000-09-30         11.0         1.0          0.0             0.0  2.484907   
2  2000-10-01          9.0         1.0          1.0             1.0  2.302585   
3  2000-10-02        